In [1]:
!pip install statsmodels

In [2]:
!pip install pandas numpy matplotlib seaborn plotly statsmodels dash

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import chardet

In [ ]:
filename = 'PurchasesFINAL12312016.csv'
with open('PurchasesFINAL12312016.csv','rb') as f:
    result = chardet.detect(f.read())
    encoding=result['encoding']
import os
os.system(f"file -i {filename}")
data = pd.read_csv('PurchasesFINAL12312016.csv', encoding=encoding, engine='python')
data.head()

In [ ]:
data.info()

In [ ]:
df = data.copy()

In [ ]:
pd.to_datetime(df['PODate'])

In [ ]:
date_col = ['PODate', 'ReceivingDate', 'InvoiceDate','PayDate']
df[date_col] = df[date_col].apply(pd.to_datetime)

In [ ]:
data.info()

In [ ]:
str_col = ['Store','Brand','Classification','VendorNumber','PONumber']
df[str_col] = df[str_col].astype(str)

In [ ]:
data.info()

In [ ]:
(df['InvoiceDate'] - df['ReceivingDate']).dt.days

In [ ]:
# Change Dtypes: Timedelta -> integer
df['LeadTime'] = (df['InvoiceDate'] - df['ReceivingDate']).dt.days

In [ ]:
df[['InvoiceDate', 'ReceivingDate', 'LeadTime']].head()

In [ ]:
df['LeadTime'].describe()

In [ ]:
df['OrderTime'] = (df['ReceivingDate'] - df['PODate']).dt.days

In [ ]:
df[['PODate','ReceivingDate','OrderTime']].head()

In [ ]:
df['OrderTime'].describe()

In [ ]:
((df['InvoiceDate'] - df['ReceivingDate']) < pd.Timedelta(0)).value_counts()

In [ ]:
df['PaymentLT'] = (df['PayDate'] - df['InvoiceDate']).dt.days

In [ ]:
df['PaymentLT'].describe()

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df['InventoryId']

In [ ]:
#### Split by '_' and expand into separate columns
df['InventoryId'].str.split('_', expand=True)

In [ ]:
df[['Split_1', 'InventoryCategory', 'Split_3']] = df['InventoryId'].str.split('_', expand=True)

In [ ]:
df.head()

In [ ]:
df = df.drop(columns=['Split_1', 'Split_3'])

In [ ]:
df.columns

In [ ]:
df['InventoryCategory'].value_counts()

In [ ]:
df['InventoryCategory'].isna().value_counts()

In [ ]:
(df['InventoryCategory'] =='').value_counts()

In [ ]:
df[df['InventoryCategory'] == ''][['InventoryId','InventoryCategory']]

In [ ]:
df1 = df.copy()

In [ ]:
df1 = df[df['InventoryCategory'] != '']

In [ ]:
print(df.shape, df1.shape)

In [ ]:
df1.select_dtypes(include=[float, int])

In [ ]:
df1.columns

In [ ]:
df1.select_dtypes(include=['datetime']).columns.to_list()

In [ ]:
numerical_features =  df1.select_dtypes(include=[float, int]).columns.to_list()
date_features = df1.select_dtypes(include=['datetime']).columns.to_list()
categorical_features =  df1.columns.difference(numerical_features + date_features).to_list()

print(numerical_features)
print(date_features)
print(categorical_features)

In [ ]:
print(len(numerical_features + date_features + categorical_features))
print(len(df1.columns))

In [ ]:
df1['LeadTime'].hist(bins=20)
plt.title('Distribution of Lead Time')
plt.xlabel('Lead Time (Days)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
sns.boxplot(x='LeadTime', data=df1)
plt.title('Lead Time Outliers')
plt.show()

In [ ]:
df2 = df1.copy()

In [ ]:
df2['LogLeadTime'] = np.log(df1['LeadTime'] + 1)

In [ ]:
df2['LogLeadTime'].hist(bins=20)
plt.title('Distribution of Lead Time: Log Transformation')
plt.xlabel('Lead Time (Days)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
df2['OrderWeekday'] = df1['PODate'].dt.weekday

In [ ]:
df2['PODate'].hist(bins=20)
plt.title('Distribution of Order Date')
plt.xlabel('Order Date')
plt.ylabel('Frequency')
plt.show()

In [ ]:
df2['OrderWeekday'].hist(bins=20)
plt.title('Distribution of Order Date-Weekday')
plt.xlabel('Order Time (Weekday)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
df1.groupby('VendorNumber').agg({
    'LeadTime': ['mean', 'min', 'max'],
    'OrderTime': ['mean', 'min', 'max'],
    'PaymentLT': ['mean',  'min', 'max']
})

In [ ]:
summary_vendor = df1.groupby('VendorNumber').agg({
                'LeadTime': ['mean', 'min', 'max'],
                'OrderTime': ['mean', 'min', 'max'],
                'PaymentLT': ['mean',  'min', 'max']
            })

In [ ]:
summary_vendor.sort_values(('OrderTime', 'mean'), ascending=False).head(5)

In [ ]:
summary_vendor.sort_values(('OrderTime', 'mean'), ascending=True).head(5)

In [ ]:
top_bad_5 = summary_vendor.sort_values(('OrderTime', 'mean'), ascending=False).head(5)
top_bad_10 = summary_vendor.sort_values(('OrderTime', 'mean'), ascending=False).head(10)


top_bad_5_index = top_bad_5.index.tolist()
top_bad_10_index = top_bad_10.index.tolist()

print(top_bad_5_index)
print(top_bad_10_index)

In [ ]:
top_good_5 = summary_vendor.sort_values(('OrderTime', 'mean'), ascending=True).head(5)
top_good_5_index = top_good_5.index.tolist()
print(top_good_5_index)

In [ ]:
print(top_bad_5_index)
filtered_top5_bad_vendor= df1[df1['VendorNumber'].isin(top_bad_5_index)]

supplier_performance = filtered_top5_bad_vendor.groupby('VendorNumber')['OrderTime'].mean().reset_index()

supplier_performance = supplier_performance.sort_values(by='OrderTime', ascending=True)

plt.bar(supplier_performance['VendorNumber'], supplier_performance['OrderTime'], color='skyblue')

plt.title('Average OrderTime by Supplier: Worst Top5')
plt.xlabel('Supplier')
plt.ylabel('Average OrderTime (Days)')

plt.yticks(ticks=range(0, int(supplier_performance['OrderTime'].max()) + 1))

plt.tight_layout()
plt.show()

In [ ]:
print(top_bad_10_index)
filtered_top10_bad_vendor= df1[df1['VendorNumber'].isin(top_bad_10_index)]

supplier_performance = filtered_top10_bad_vendor.groupby('VendorNumber')['OrderTime'].mean().reset_index()

supplier_performance = supplier_performance.sort_values(by='OrderTime', ascending=True)

plt.bar(supplier_performance['VendorNumber'], supplier_performance['OrderTime'], color='skyblue')

plt.title('Average OrderTime by Supplier: Worst Top10')
plt.xlabel('Supplier')
plt.ylabel('Average OrderTime (Days)')

plt.yticks(ticks=range(0, int(supplier_performance['OrderTime'].max()) + 1))

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
filtered_top5_bad_vendor.boxplot(column='OrderTime', by='VendorNumber', grid=False)

plt.title('Order Time Distribution by Supplier:Worst Top10')
plt.suptitle('')
plt.xlabel('Supplier')
plt.ylabel('OrderTime (Days)')

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
filtered_top10_bad_vendor.boxplot(column='OrderTime', by='VendorNumber', grid=False)

plt.title('Order Time Distribution by Supplier: Worst Top10')
plt.suptitle('')
plt.xlabel('Supplier')
plt.ylabel('OrderTime (Days)')

plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns

plt.figure(figsize=(8, 6))
sns.violinplot(x='VendorNumber', y='OrderTime', data=filtered_top10_bad_vendor)

plt.title('Order Time Distribution by Supplier (Violin Plot): Worst Top10')
plt.xlabel('Supplier')
plt.ylabel('Order Time (Days)')

plt.tight_layout()
plt.show()

In [ ]:
filtered_top10_bad_vendor.info()

In [ ]:
import seaborn as sns
import pandas as pd

filtered_top10_bad_vendor['PODate'] = pd.to_datetime(filtered_top10_bad_vendor['PODate'])
lead_time_heatmap_data = filtered_top10_bad_vendor.pivot_table(index=filtered_top10_bad_vendor['PODate'].dt.to_period('M'), columns='VendorNumber', values='OrderTime', aggfunc='mean')

plt.figure(figsize=(10, 6))
sns.heatmap(lead_time_heatmap_data, cmap='coolwarm', annot=True, fmt=".1f")

plt.title('Supplier Lead Time Heatmap Over Time')
plt.xlabel('Supplier')
plt.ylabel('Month')

plt.tight_layout()
plt.show()

In [ ]:
print(top_good_5_index)
filtered_top5_good_vendor= df1[df1['VendorNumber'].isin(top_good_5_index)]

supplier_performance = filtered_top5_good_vendor.groupby('VendorNumber')['OrderTime'].mean().reset_index()

supplier_performance = supplier_performance.sort_values(by='OrderTime', ascending=True)

plt.bar(supplier_performance['VendorNumber'], supplier_performance['OrderTime'], color='skyblue')

plt.title('Average OrderTime by Supplier')
plt.xlabel('Supplier')
plt.ylabel('Average OrderTime (Days)')

plt.yticks(ticks=range(0, int(supplier_performance['OrderTime'].max()) + 1))

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
filtered_top5_good_vendor.boxplot(column='OrderTime', by='VendorNumber', grid=False)

plt.title('Order Time Distribution by Supplier')
plt.suptitle('')
plt.xlabel('Supplier')
plt.ylabel('OrderTime (Days)')

plt.tight_layout()
plt.show()

In [ ]:
summary_store = df1.groupby('Store').agg({
                'LeadTime': ['mean', 'min', 'max'],
                'OrderTime': ['mean', 'min', 'max'],
                'PaymentLT': ['mean',  'min', 'max']
            })

In [ ]:
summary_store.sort_values(('PaymentLT', 'mean'), ascending=False).head(5)

In [ ]:
summary_store.sort_values(('PaymentLT', 'mean'), ascending=True).head(5)

In [ ]:
summary_inventoryCat = df1.groupby('InventoryCategory').agg({
                'LeadTime': ['mean', 'min', 'max'],
                'OrderTime': ['mean', 'min', 'max'],
                'PaymentLT': ['mean',  'min', 'max']
            })

In [ ]:
summary_inventoryCat.sort_values(('OrderTime', 'mean'), ascending=True).head(5)

In [ ]:
summary_inventoryCat.sort_values(('OrderTime', 'mean'), ascending=False).head(5)

In [ ]:
podate_att = df1.groupby(['PODate', 'Classification']).apply(lambda x: x['PODate'].count()).reset_index(name='Counts')

fig = px.line(podate_att, x='PODate', y='Counts', color='Classification', title='Purchase Order Counts by Date and Classification')

fig.show()

In [ ]:
df1[(df1['InventoryCategory'] == 'ABERDEEN') & (df1['PODate'] == '2016-06-27')]

In [ ]:
numeric_df1 = df1[numerical_features]

correlation_matrix = numeric_df1.corr()

print(correlation_matrix)

In [ ]:
pd.DataFrame(correlation_matrix).to_clipboard()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

numeric_df1 = df1[numerical_features]

correlation_matrix = numeric_df1.corr()

plt.figure(figsize=(10, 8))

sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', linewidths=0.5)

plt.title('Correlation Matrix of Numerical Features')
plt.show()

In [ ]:
filtered_top5_good_vendor['Classification'].value_counts()

In [ ]:
filtered_top5_good_vendor['Classification'].value_counts()

In [ ]:
filtered_top5_good_vendor['RollingMeanLeadTime'] = filtered_top5_good_vendor['LeadTime'].rolling(window=10).mean()
plt.plot(filtered_top5_good_vendor['PODate'], filtered_top5_good_vendor['RollingMeanLeadTime'])
plt.title('Rolling Average of Lead Time')
plt.xlabel('Date')
plt.ylabel('Lead Time (Days)')
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(10, 6))


for inventory_class in filtered_top5_good_vendor['Classification'].unique():
    inventory_data = filtered_top5_good_vendor[filtered_top5_good_vendor['Classification'] == inventory_class]
    plt.plot(inventory_data['PODate'], inventory_data['RollingMeanLeadTime'], label=f'Classification: {inventory_class}')


plt.title('Rolling Average of Lead Time by Classification')
plt.xlabel('Date')
plt.ylabel('Rolling Mean Lead Time (Days)')
plt.legend(title='Classification')


plt.tight_layout()
plt.show()

In [ ]:
podate_att = df1.groupby(['PODate', 'InventoryCategory']).apply(lambda x: x['PODate'].count()).reset_index(name='Counts')


fig = px.line(podate_att, x='PODate', y='Counts', color='InventoryCategory', title='Purchase Order Counts by Date and InventoryCategory')


fig.show()

In [ ]:
filtered_top5_bad_vendor['RollingMeanLeadTime'] = filtered_top5_bad_vendor['LeadTime'].rolling(window=10).mean()
plt.plot(filtered_top5_bad_vendor['PODate'], filtered_top5_bad_vendor['RollingMeanLeadTime'])
plt.title('Rolling Average of Lead Time: Worst Top5 Vendor')
plt.xlabel('Date')
plt.ylabel('Lead Time (Days)')
plt.xticks(rotation=45)
plt.show()

In [ ]:
filtered_top5_bad_vendor['RollingMeanLeadTime'].describe()

In [ ]:
plt.figure(figsize=(10, 6))
for inventory_cat in filtered_top5_bad_vendor['InventoryCategory'].unique():
    inventory_data = filtered_top5_bad_vendor[filtered_top5_bad_vendor['InventoryCategory'] == inventory_cat]
    plt.plot(inventory_data['PODate'], inventory_data['RollingMeanLeadTime'], label=f'InventoryCategory: {inventory_cat}')

plt.title('Rolling Average of Lead Time by InventoryCategory')
plt.xlabel('Date')
plt.ylabel('Rolling Mean Lead Time (Days)')
plt.legend(title='InventoryCategory')

plt.tight_layout()
plt.show()

In [ ]:
filtered_top5_good_vendor['RollingMeanLeadTime'] = filtered_top5_good_vendor['LeadTime'].rolling(window=10).mean()
plt.plot(filtered_top5_good_vendor['PODate'], filtered_top5_good_vendor['RollingMeanLeadTime'])
plt.title('Rolling Average of Lead Time: Best Top5 Vendor')
plt.xlabel('Date')
plt.ylabel('Lead Time (Days)')
plt.xticks(rotation=45)
plt.show()
plt.figure(figsize=(10, 6))


for inventory_cat in filtered_top5_good_vendor['InventoryCategory'].unique():
    inventory_data = filtered_top5_good_vendor[filtered_top5_good_vendor['InventoryCategory'] == inventory_cat]
    plt.plot(inventory_data['PODate'], inventory_data['RollingMeanLeadTime'], label=f'InventoryCategory: {inventory_cat}')

plt.title('Rolling Average of Lead Time by InventoryCategory')
plt.xlabel('Date')
plt.ylabel('Rolling Mean Lead Time (Days)')
plt.legend(title='InventoryCategory')

plt.tight_layout()
plt.show()